# Split mini representative dataset

In [1]:
from createCSV import TrainTest
import config

if __name__ == "__main__":
    csv = TrainTest(
        base_dir = config.BASE_DIR,
        all_csv = config.ALL_CSV,
        train_csv = config.TRAIN_CSV,
        test_csv = config.TEST_CSV,
        label_column = config.LABEL_COLUMN,
        test_size = config.TEST_SIZE,
        seed = config.SEED)
    csv.set_all_seeds()
    csv.df()

Train set length:
772
Test set length:
228
Train set label distribution:
0    390
1    382
Name: label, dtype: int64
Test set label distribution:
1    118
0    110
Name: label, dtype: int64


In [2]:
from createCSV import TrainVal
import config

if __name__ == "__main__":
    csv1 = TrainVal(
        orig_train_csv = config.TRAIN_CSV,
        train_csv = config.TRAIN_CSV_1,
        val_csv = config.VAL_CSV_1,
        label_column = config.LABEL_COLUMN,
        val_size = config.VAL_SIZE,
        seed = config.SEED)
    csv1.set_all_seeds()
    csv1.df()

Original train set length:
772
New train set length and first n rows:
621
                                          file_name  label   parcel_id
0  21-0176367_F87DA57823CDD81CE0530EEE260ADB63.jpeg      0  21-0176367
1  21-0176367_F87DA57823FDD81CE0530EEE260ADB63.jpeg      0  21-0176367
2  21-0176564_F87DA57819B5D81CE0530EEE260ADB63.jpeg      0  21-0176564
3  21-0176564_F87DA57819B6D81CE0530EEE260ADB63.jpeg      0  21-0176564
4  21-0178213_F87DA5782399D81CE0530EEE260ADB63.jpeg      0  21-0178213
Val set length and first n rows:
151
                                            file_name  label   parcel_id
120  21-0179791_F87DA5781D9FD81CE0530EEE260ADB63.jpeg      0  21-0179791
161  21-0184022_F87DA5781E3CD81CE0530EEE260ADB63.jpeg      0  21-0184022
162  21-0184022_F87DA5781E4DD81CE0530EEE260ADB63.jpeg      0  21-0184022
163  21-0184022_F87DA5781E54D81CE0530EEE260ADB63.jpeg      0  21-0184022
164  21-0184022_F87DA5781E9DD81CE0530EEE260ADB63.jpeg      0  21-0184022
Train set label distribut

# Train model on mini representative dataset

In [1]:
# Train model
from model import NN
from customDataModule import CustomDataModule
import config
import pytorch_lightning as pl
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch import seed_everything
from lightning.pytorch.callbacks import ModelCheckpoint
import torch
import torchvision.models as models
import torch.nn as nn

seed_everything(
    42, workers=True
)  # By setting workers=True in seed_everything(), Lightning derives unique seeds across all dataloader workers and processes for torch, numpy and stdlib random number generators. When turned on, it ensures that e.g. data augmentations are not repeated across workers.



if __name__ == "__main__":
    
    model = models.vgg16(weights='VGG16_Weights.DEFAULT')
    
    logger = TensorBoardLogger("tb_logs", name="representative") # tb_logs is the folder, name is the name of the experiment/model
    model = NN(
        model=model,
        input_shape=config.INPUT_SHAPE,
        num_classes=config.NUM_CLASSES,
        learning_rate=config.LEARNING_RATE,
    )  # .to(device)
    dm = CustomDataModule(
        data_dir=config.DATA_DIR,
        train_csv=config.TRAIN_CSV_1,
        val_csv=config.VAL_CSV_1,
        test_csv=config.TEST_CSV,
        batch_size=config.BATCH_SIZE,
        num_workers=config.NUM_WORKERS,
    )
    

    
    trainer = pl.Trainer(
        logger=logger, # PyTorch lightning will automatically know what we are logging by looking at our model.py logs
        accelerator=config.ACCELERATOR,
        devices=config.DEVICES,
        min_epochs=config.MIN_EPOCHS,
        max_epochs=5,#config.MAX_EPOCHS,
        deterministic=config.DETERMINISTIC,
#        callbacks=[ModelCheckpoint(monitor="val_loss", mode="min")],
    )
      # deterministic ensures random seed reproducibility

    trainer.fit(model, dm)  # it will automatically know which dataloader to use

# A general place to start is to set num_workers equal to the number of CPU cores on that machine. You can get the number of CPU cores in python using os.cpu_count(), but note that depending on your batch size, you may overflow RAM memory.


Global seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type                | Params
--------------------------------------------------------
0 | model           | VGG                 | 138 M 
1 | classifier      | Linear              | 2.0 K 
2 | loss_fn         | CrossEntropyLoss    | 0     
3 | train_acc       | MulticlassAccuracy  | 0     
4 | val_acc         | MulticlassAccuracy  | 0     
5 | train_recall    | MulticlassRecall    | 0     
6 | train_precision | MulticlassPrecision | 0     
7 | train_f1score   | MulticlassF1Score   | 0     
--------------------------------------------------------
2.0 K     Trainable params
138 M     Non-trainable params
138 M     Total params
553.438   Total estimated model params size (MB)


C:\Users\kaspe\.conda\envs\LBST\lib\site-packages\pytorch_lightning\loops\fit_loop.py:280: PossibleUserWarning: The number of training batches (20) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 0: 100%|██████████| 20/20 [00:10<00:00,  1.99it/s, v_num=4]
Validation: 0it [00:00, ?it/s]
Epoch 1: 100%|██████████| 20/20 [00:10<00:00,  1.95it/s, v_num=4, train_loss=0.741, train_accuracy=0.626, train_recall=0.626, train_precision=0.626, train_f1score=0.626]
Validation: 0it [00:00, ?it/s]
Epoch 2: 100%|██████████| 20/20 [00:10<00:00,  1.92it/s, v_num=4, train_loss=0.490, train_accuracy=0.765, train_recall=0.765, train_precision=0.765, train_f1score=0.765]
Validation: 0it [00:00, ?it/s]
Epoch 3: 100%|██████████| 20/20 [00:10<00:00,  1.93it/s, v_num=4, train_loss=0.460, train_accuracy=0.784, train_recall=0.784, train_precision=0.784, train_f1score=0.784]
Validation: 0it [00:00, ?it/s]
Epoch 4: 100%|██████████| 20/20 [00:09<00:00,  2.02it/s, v_num=4, train_loss=0.391, train_accuracy=0.820, train_recall=0.820, train_precision=0.820, train_f1score=0.820]
Validation: 0it [00:00, ?it/s]
Epoch 4: 100%|██████████| 20/20 [00:15<00:00,  1.31it/s, v_num=4, train_loss=0.424, train_accuracy=

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 20/20 [00:16<00:00,  1.19it/s, v_num=4, train_loss=0.424, train_accuracy=0.815, train_recall=0.815, train_precision=0.815, train_f1score=0.815]


In [17]:
preds[1]

tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
        0, 1, 1, 0, 0, 1, 0, 1])

In [19]:
for y in dm.train_dataloader():  
    break
y = y
y[1]

tensor([1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1,
        1, 0, 0, 0, 0, 1, 0, 0])

In [20]:
type(y)

list

TypeError: float() argument must be a string or a real number, not 'list'

In [ ]:
for batch, batch_idx in model.training_step(batch, batch_idx):
    break
print(y, preds)

NameError: name 'batch' is not defined

In [6]:
for y in dm.train_dataloader():  
    break
print(y)

[tensor([[[[ 0.8447,  1.3413,  1.7694,  ...,  1.5297,  1.5468,  1.5468],
          [ 0.7762,  1.1529,  1.7180,  ...,  1.5297,  1.5297,  1.5468],
          [ 0.6221,  0.9474,  1.6324,  ...,  1.5297,  1.5297,  1.5468],
          ...,
          [-1.0733, -0.8335, -0.5767,  ..., -1.2445, -1.0904, -1.0390],
          [-0.7650, -0.7479, -0.6452,  ..., -1.3815, -1.1760, -1.1589],
          [-0.6281, -0.6965, -0.5596,  ..., -1.3987, -1.3473, -1.0219]],

         [[ 1.1506,  1.5882,  1.9559,  ...,  1.7283,  1.7458,  1.7458],
          [ 0.9930,  1.3606,  1.9384,  ...,  1.7283,  1.7283,  1.7458],
          [ 0.8004,  1.0980,  1.8333,  ...,  1.7283,  1.7283,  1.7458],
          ...,
          [-1.1429, -0.9328, -0.7052,  ..., -1.0378, -0.8978, -0.8978],
          [-0.8452, -0.8452, -0.7577,  ..., -1.1954, -0.9678, -0.9678],
          [-0.7227, -0.8102, -0.6702,  ..., -1.2304, -1.0903, -0.8102]],

         [[ 1.5245,  1.9254,  2.2914,  ...,  2.1520,  2.1694,  2.1694],
          [ 1.3328,  1.7163, 

In [11]:
for preds in dm.train_dataloader():  
    break
print(preds)

[tensor([[[[-1.0390, -1.0219, -0.9877,  ...,  0.3652,  0.3823,  0.3823],
          [-1.0219, -1.0048, -0.9877,  ...,  0.3823,  0.3823,  0.3994],
          [-1.0219, -1.0048, -0.9705,  ...,  0.3652,  0.3823,  0.3994],
          ...,
          [-0.1314,  0.3481,  0.2453,  ...,  0.0912, -0.1314, -0.6965],
          [ 0.0227,  0.0227,  0.6906,  ..., -0.1999,  0.0741, -0.1486],
          [-0.2171, -0.3541,  0.5022,  ..., -0.1828, -0.0801,  0.0569]],

         [[-0.0049,  0.0301,  0.0476,  ...,  0.9405,  0.9405,  0.9230],
          [ 0.0126,  0.0476,  0.0651,  ...,  0.9580,  0.9755,  0.9580],
          [ 0.0126,  0.0476,  0.0826,  ...,  0.9930,  0.9930,  0.9755],
          ...,
          [-0.6702, -0.1975, -0.2150,  ..., -0.2325, -0.3725, -0.8452],
          [-0.4426, -0.4776,  0.2402,  ..., -0.4951, -0.2150, -0.3901],
          [-0.6001, -0.6176,  0.0301,  ..., -0.4951, -0.4601, -0.2675]],

         [[ 1.4200,  1.4722,  1.4722,  ...,  1.9603,  1.9603,  1.9254],
          [ 1.4722,  1.5071, 

In [9]:
for labels in dm.train_dataloader():  
    break
print(labels)

[tensor([[[[ 0.8447,  1.3413,  1.7694,  ...,  1.5297,  1.5468,  1.5468],
          [ 0.7762,  1.1529,  1.7180,  ...,  1.5297,  1.5297,  1.5468],
          [ 0.6221,  0.9474,  1.6324,  ...,  1.5297,  1.5297,  1.5468],
          ...,
          [-1.0733, -0.8335, -0.5767,  ..., -1.2445, -1.0904, -1.0390],
          [-0.7650, -0.7479, -0.6452,  ..., -1.3815, -1.1760, -1.1589],
          [-0.6281, -0.6965, -0.5596,  ..., -1.3987, -1.3473, -1.0219]],

         [[ 1.1506,  1.5882,  1.9559,  ...,  1.7283,  1.7458,  1.7458],
          [ 0.9930,  1.3606,  1.9384,  ...,  1.7283,  1.7283,  1.7458],
          [ 0.8004,  1.0980,  1.8333,  ...,  1.7283,  1.7283,  1.7458],
          ...,
          [-1.1429, -0.9328, -0.7052,  ..., -1.0378, -0.8978, -0.8978],
          [-0.8452, -0.8452, -0.7577,  ..., -1.1954, -0.9678, -0.9678],
          [-0.7227, -0.8102, -0.6702,  ..., -1.2304, -1.0903, -0.8102]],

         [[ 1.5245,  1.9254,  2.2914,  ...,  2.1520,  2.1694,  2.1694],
          [ 1.3328,  1.7163, 

In [6]:
trainer.validate(model, datamodule=dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation DataLoader 0: 100%|██████████| 5/5 [00:01<00:00,  4.16it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       val_accuracy        │     0.748344361782074     │
│         val_loss          │    0.5354300737380981     │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': 0.5354300737380981, 'val_accuracy': 0.748344361782074}]

In [7]:
trainer.test(model, datamodule=dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 8/8 [00:01<00:00,  4.50it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.3434378206729889     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.3434378206729889}]

In [3]:
trainer.predict(model, datamodule=dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


MisconfigurationException: `predict_dataloader` must be implemented to be used with the Lightning Trainer